In [1]:
%reset-f
%load_ext autoreload
%autoreload
%matplotlib inline

In [2]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy as sp
from scipy.optimize import curve_fit
from tools import *



In [56]:
country_name = "peru"
outdir = "out_"+country_name+"/"
os.makedirs(outdir,exist_ok=True)
input_dir = "input/"+country_name+"/"

### Piecewise linear trajectory

In [57]:
def traj(x,s,t,a,m):
    #print("in",(x,s,t,u,a,m))
    u=x[-1]#forces u to be last value, this is equivalent siince we still chose a
    
    s=max(s,x[0])
    a=max(a,0)
    
    return extrap(x,[x[0],s,t,u],[0,0,a,max(m,a)]) #forces positive slope

### Data

In [58]:
#Reads abatement the data
df_in=pd.read_csv(input_dir+"abat.csv").set_index("year")

#Rescales the data
power = (np.floor(np.log10(df_in.max().max())/3))*3
print("scaling at x10^-"+str(power))
abat_xl=df_in * 10**(-power);

year=df_in.index.values
print(year)

#Cost data
macs  = pd.read_csv(input_dir+"mac_names.csv").set_index("measure")
    

scaling at x10^-0.0
[2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024
 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039
 2040 2041 2042 2043 2044 2045 2046 2047 2048 2049 2050]


In [59]:
#Output dataframe
df = pd.DataFrame(index=df_in.columns,columns=["speed","MAC","potential","t","growth"])
df["MAC"]=macs

In [61]:
df

,speed,MAC,potential,t,growth
ENE01,NaN,62.6,NaN,NaN,NaN
ENE02,NaN,108.1,NaN,NaN,NaN
ENE03,NaN,151.0,NaN,NaN,NaN
ENE04,NaN,101.1,NaN,NaN,NaN
ENE05,NaN,54.4,NaN,NaN,NaN
ENE06,NaN,-60.4,NaN,NaN,NaN
ENE07,NaN,-12.7,NaN,NaN,NaN
ENE08,NaN,111.3,NaN,NaN,NaN
ENE09,NaN,509.4,NaN,NaN,NaN
ENE10,NaN,835.9,NaN,NaN,NaN


### Loop on measures

In [62]:
for j in df.index:
#for j in ["Restauration"]:
    y=abat_xl.ix[:,j].interpolate() #fills missing values by interpolating

    #guess
    s_ = year[3];
    t_ = year[15];
    a_ = y[t_];
    m_ = y[year[-1]]
    
    #fit
    (s,t,a,m),foo =curve_fit(traj, year, y, maxfev=int(1e4), p0 = [s_, t_,a_,m_])
   
    #makes sure start is before t
    s,t = min(s,t),max(s,t)
    
    #plot
    plt.scatter(year,y,c="red")
    plt.plot(year,traj(year,s,t,a,m))
    
    u=year[-1]
    
    #speeds
    if abs(s-t)<2: #case with a jump
        va= a
    else:
        va = a/(t-s)
        
    vb = (m-a)/(u-t)
    
    if va>vb: #classic case
        df.ix[j,"t"]=t
        df.ix[j,"speed"]=va
        df.ix[j,"growth"]=vb
        df.ix[j,"potential"]=a
    else:
        v=m/(u-s)
        df.ix[j,"t"]=s
        df.ix[j,"speed"]=v
        df.ix[j,"growth"]=v
        df.ix[j,"potential"]=0
        
        plt.plot(year,traj(year,s,s,0,v*(u-s)),color="green")

    #saves figures and clears
    plt.savefig(outdir+j+".png")
    plt.close()
    

In [63]:
df

,speed,MAC,potential,t,growth
ENE01,0.2401269,62.6,2.796028,2025.113,0.1579011
ENE02,0.2725366,108.1,3.538551,2026.242,0.1899139
ENE03,0.3222281,151.0,3.961136,2025.581,0.2203562
ENE04,0.07504114,101.1,0,2011.102,0.07504114
ENE05,0.2857428,54.4,3.057048,2024.132,0.1480607
ENE06,0.7077897,-60.4,0,2012.825,0.7077897
ENE07,4.704875,-12.7,4.704875,2018.382,0.2517073
ENE08,0.0607296,111.3,0.6731558,2025,0.03638935
ENE09,0.01909091,509.4,0.01909091,2017.901,-2.559847e-05
ENE10,0,835.9,0,2013,0


In [114]:
df.to_csv(outdir+"macinert_inputs.tsv",sep="\t")